## Tutorial: Create an OpenFlow Network and Servers

This notebook demonstrates how to create isolated VLANs and attach servers.

#### Modules 

- [Reserve a Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

In [1]:
import json
import os
import chi

from chi.server import *
from chi.lease import *
from chi.network import *

from datetime import datetime, timedelta
from dateutil import tz
import numbers
import sys
import time

#Config with your project and site
#chi.set('project_name', 'ExoGENI@Chameleon') # Replace with your project name
#chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

#Config with your project and site
chi.set('project_name', 'CH-822154') # Replace with your project name
chi.use_site('CHI@TACC')


#Insert keypair name
key_name = 'my_chameleon_key'  # Change to your keypair

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server attributes
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_skylake"
server_count=2
physical_network="physnet1"
of_controller_ip="129.114.108.132"
of_controller_port = "6653"
vswitch_name = username+"SDN"
DEFAULT_CIDR = '192.168.1.0/24'

#### Create a Lease

In [2]:
#add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network,of_controller_ip=of_controller_ip, of_controller_port=of_controller_port, vswitch_name=vswitch_name)

create_network(network_name, of_controller_ip=of_controller_ip, of_controller_port=of_controller_port,
                   vswitch_name=None, provider='physnet1',
                   port_security_enabled=True)

Unauthorized: 401-{'error': {'message': 'The request you have made requires authentication.', 'code': 401, 'title': 'Unauthorized'}}
Neutron server returns request_ids: ['req-4e87564f-b9cd-4c8a-8d3b-6a8b4293fb4b']

In [ ]:
#Get the lease by name
lease = get_lease(lease_name)
    
#Print the lease info
print(json.dumps(lease, indent=2))

#### Get the Reservations

In [ ]:
#Get the lease by name
lease = get_lease(lease_name)

compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', lease['reservations']))[0]['id']
network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', lease['reservations']))[0]['id']
floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("network_reservation_id: " + network_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

#### Configute the Network

In [ ]:
network = get_network(network_name)
network_id = network['id']

print('Network ID: ' + str(network_id))

In [ ]:
subnet = create_subnet(subnet_name, network_id, cidr=DEFAULT_CIDR, gateway_ip=None)
router = create_router(router_name, gw_network_name='public')
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

#### Start the Server

In [ ]:
#create the server
server = create_server(server_name, 
                       compute_reservation_id, 
                       key_name=key_name, 
                       network_id=network_id, 
                       network_name=network_name, 
                       nics=[], 
                       image_id=None, 
                       image_name=image_name, 
                       flavor_id=None, 
                       flavor_name=flavor_name, 
                       count=1)

#### Associate the Floating IP   (TODO: need to find floating_ip from the reservation that was just made)

In [ ]:
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

#### Delete Server

In [ ]:
delete_server(server['id'])

#### De-configure Network

In [ ]:
try:
    result = remove_subnet_from_router(router['id'], subnet['id'])
except Exception as e:
    print("remove_subnet_from_router error:" + str(e))
    pass

try:
    result = delete_router(router['id'])
except Exception as e:
    print("delete_router error: " + str(e))
    pass

try:
    result = delete_subnet(subnet['id'])
except Exception as e:
    print("delete_subnet error: " + str(e))
    pass

try:
    result = delete_network(network_id)
except Exception as e:
    print("delete_network error: " + str(e))
    pass

#### Release Lease

In [ ]:
delete_lease(lease_name)